# Import modules

In [257]:
!pip install graphene
import graphene
import json
import sys, io
import pandas as pd
import numpy as np
import importlib
import os
import pathlib
import requests
import torch

bundle_root = pathlib.Path(os.environ.get('LABS_BUNDLE_ROOT', '/labs'))
sys.path.append(str(bundle_root / 'functions/index_by_nationality'))

In [258]:
query_metadata = """query metadata {nationalityList, jobList}"""
query_data = """query data {movieList(nationality: "Russian", job: "Director")
                {
                title
                revenue
                }}"""

# Test handler functions

In [259]:
import handler 
importlib.reload(handler);

In [260]:
handler.handle(query_data)

'{"movieList": [{"title": "Jurassic World", "revenue": 1513528810}, {"title": "The Fate of the Furious", "revenue": 1238764765}, {"title": "Captain America: Civil War", "revenue": 1153304495}]}'

# Deploy function

### cheat sheet

In [6]:
# labs bundle deploy staging (deploy to staging)
# labs bundle build classify_name (build locally for debugging)
# labs function run sunburst -d '{"date_range": [20180201,20180401], "max_level":4, "threshold":1000, "operating_system": "all"}' (example local call)

In [ ]:
#labs bundle build classify_name && docker run -it --rm --name function_test -p 5000:5000 classify_name python index.py

# Test function 

In [7]:
!curl -d '{"max_level":8,"threshold":10000,"date_range":["2018-02-17","2018-04-18"]}' https://labs.ctmo.io/21b899ae-3eba-56a7-97da-fa26c050d1c5/v1/function/08fb1d62-066a-42d6-bfb4-12d127c5bfd2/sunburst

<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx/1.13.12</center>
</body>
</html>


In [8]:
%%time
deployed_output = requests.post('https://labs.stg.contiamo.io/21b899ae-3eba-56a7-97da-fa26c050d1c5/v1/function/6d02283a-7896-4f42-9ae2-a18cccfd9318/sunburst',
                                data= json.dumps({'date_range': ["2018-04-03","2018-04-06"],
                                                  'max_level': 8, 
                                                  'threshold': 1000,
                                                  'operating_system': 'all',
                                                  'browser': 'all'})).json()

CPU times: user 21 ms, sys: 2.63 ms, total: 23.7 ms
Wall time: 848 ms


deployed_output

In [9]:
deployed_output['metadata']

{'current_date': '2018-08-24',
 'date_range': ['2017-09-01', '2018-08-23'],
 'load_time (ms)': 1741,
 'os_values': ['Android', 'iOS', 'Other', 'BlackBerry'],
 't_function_call (ms)': 113,
 't_package_import (ms)': 469,
 't_parse_input (ms)': 2,
 'version': '1.7'}

In [10]:
deployed_output['sunburst_data']['data']['value']

'182083'

### Compare

In [165]:
output['sunburst_data'] == deployed_output['sunburst_data']

True